In [6]:
def problem1(datafile,compressed_file):
    ndrv = pow(2,15)-1 
    import netCDF4
    import numpy as np
    nc1=netCDF4.Dataset(datafile)
    nc2 = netCDF4.Dataset(compressed_file,"w")
    # read dimensions from source and create in destination file
    for key,value in nc1.dimensions.items():
        dumm=nc2.createDimension(key,len(value))
    # read variables from source and create in destination file
    for key in nc1.variables:
      # read data
        vobj = nc1.variables[key]
        data = vobj[:]
      #data = np.float64(vobj[:])
      # calculate scale_factor and add_offset
        xmin = data.min()
        xmax = data.max()
        scale_factor = (xmax-xmin)/ndrv
        add_offset = (xmin+xmax)/2.0
        #print(xmin, xmax, scale_factor, add_offset)
        data_pck = (data.copy()-add_offset)/scale_factor

      # write to the destination file
      # if you put add_offset and scale_factor before writing data to disk then the data will be wrong in the file

        temp = nc2.createVariable(key, 'h', nc1.variables[key].dimensions)
              # add source attributes to destination
        for i in vobj.ncattrs():
            dumm = nc2.variables[key]
            dumm.setncattr(i, vobj.getncattr(i)) 
        #temp = nc2.createVariable(key, nc1.variables[key].dtype.str, nc1.variables[key].dimensions)
        #temp[:] = np.int16(data_pck)
        temp[:]=data_pck
        #temp.scale_factor = np.float32(scale_factor)
        temp.scale_factor = scale_factor
        #temp.add_offset = np.float32(add_offset)
        temp.add_offset=add_offset
        #temp[:] = np.uint16(data_pck)
        #temp = nc2.createVariable(key, nc1.variables[key].dtype.str, nc1.variables[key].dimensions)
        temp[:] = data_pck


    # cllose files
    nc1.close()
    nc2.close()

In [9]:
problem1("input.nc","input_compressed.nc")

In [33]:
def problem2(datafile,reduced_datafile,n):
    import netCDF4
    # open source file
    nc1 = netCDF4.Dataset(datafile, 'r')
    # open destination file
    nc2 = netCDF4.Dataset(reduced_datafile, 'w')
    # read dimensions from source and create in destination file
    for key,value in nc1.dimensions.items():
        dumm=nc2.createDimension(key,len(value)/2)
    for key in nc1.variables:
      # read data
        vobj = nc1.variables[key]
        data = vobj[:].copy()
        temp = nc2.createVariable(key, nc1.variables[key].dtype.str, nc1.variables[key].dimensions)
        # add source attributes to destination
        for i in vobj.ncattrs():
            dumm = nc2.variables[key]
            dumm.setncattr(i, vobj.getncattr(i)) 
        if  data.ndim==3:
            temp[:]=data[::n,::n,::n]
        else:
            temp[:]=data[::n,::n]
    nc2.close()
    nc1.close()

In [34]:
problem2("input.nc","reduced_input.nc",2)